Preparing data for analysis.

We are going to work with data provided by a company that connects different clients to different firms. More specifically, the company allows clients to book (schedule) visits to firms in order to learn more about their production processes and final products. Each booking has an unique identification ('b_id') which is used as index in the data structure. For confidentiality reason, some information cannot and will not be made public. To start, let's import the relevant modules.

In [1]:
## Relevant modules.
import os
import csv
import numpy as np
import pandas as pd

The original data was stored in four different sheets inside a microsoft excel file named 'data_original'. That excel file was lightly organized and saved as a microsoft excel file named 'data_organized'. From 'data_organized' came three csv files: 'bookings_questions_data', 'bookings_addons_data' and 'main_data'. From the internet came other three csv files with necessary additional information: 'countries_data', 'us_regions_data' and 'us_zipcodes_data'. In summary, we are going to work with the following six csv files in the 'data' directory:

- countries_data.csv
- us_regions_data.csv
- us_zipcodes_data.csv
- bookings_questions_data.csv
- bookings_addons_data.csv
- main_data.csv

In particular, the source of 'us_zipcodes_data.csv' is https://simple.wikipedia.org/wiki/List_of_ZIP_Code_prefixes.

First, we are going to work each file individually, starting with 'countries_data.csv', which contains the abbreviation code and the full name of several countries. The end product of the code cell will be:

- country_dic: dictionary with abbreviation code (key) and full name (value) for each country.
- country_indic: inverted 'country_dic' dictionary.

In [2]:
## Reading countries_data.csv as pandas dataframe countries_df 
countries_df = pd.read_csv('data/countries_data.csv',
                           index_col='country_abb', 
                           header=0, 
                           low_memory=False,
                           keep_default_na=False,
                           na_values='')

## Checking for completely empty rows.
if len(countries_df)!=len(countries_df.dropna(how='all')):
    print('Attention! There are completely empty rows!')

## Lower case for 'country'.
countries_df['country'] = countries_df['country'].str.lower()

## Creating dictionary and inverted dictionary.
country_dic = countries_df['country'].to_dict()
country_indic = {j: i for i, j in country_dic.items()}

Next, we will work with 'us_regions_data.csv', which contains the abbreviation code and the full name of all US states and territories. The end product of the code cell will be:

- us_region_dic: dictionary with abbreviation code (key) and full name (value) for each US state or territory.
- us_region_indic: inverted 'us_region_dic' dictionary.

In [3]:
## Reading us_regions_data.csv as pandas dataframe usregions_df.
usregions_df = pd.read_csv('data/us_regions_data.csv', 
                     index_col='region_abb', 
                     header=0, 
                     low_memory=False,
                     keep_default_na=False,
                     na_values='')

## Checking for completely empty rows.
if len(usregions_df)!=len(usregions_df.dropna(how='all')):
    print('Attention! There are completely empty rows!')

## Lower case and no left or right spaces for 'region'.
usregions_df['region'] = usregions_df['region'].str.lower()
usregions_df['region'] = usregions_df['region'].str.strip()

## Creating dictionary and inverted dictionary.
us_region_dic = usregions_df['region'].to_dict()
us_region_indic = {j: i for i, j in us_region_dic.items()}

Next, we will work with 'us_zipcodes_data.csv', which contains the zipcode prefixes and the abbreviation code of US states and territories. The end product of the code cell will be:

- us_zipcode_dic: dictionary with zipcode prefixes (key) and abbreviation code (value) of US states or territories.

In [4]:
## Reading us_zipcodes_data.csv as pandas dataframe uszipcodes_df.
uszipcodes_df = pd.read_csv('data/us_zipcodes_data.csv',  
                            converters={'zip_prefix': lambda x: str(x)},
                            header=0, 
                            low_memory=False,
                            keep_default_na=False,
                            na_values='')

## Checking for completely empty rows.
if len(uszipcodes_df)!=len(uszipcodes_df.dropna(how='all')):
    print('Attention! There are completely empty rows!')

## Setting 'zip_prefix' as index.
uszipcodes_df.set_index('zip_prefix', inplace=True)

## Creating dictionary.
us_zipcode_dic = uszipcodes_df['region'].to_dict()

Next, we will work with 'bookings_questions_data.csv'. This file contains extra four categorical questions associated with some bookings. They will be named as questions 'q4' to 'q7'. The end product of the code cell will be:

- questions_df: dataframe with booking id and columns for questions 'q4' to 'q7'.
- q4_dic: dictionary with 'q4' and numerical codes (0 to 1) as keys, and descriptions as values. 
- q5_dic: dictionary with 'q5' and numerical codes (1 to 3) as keys, and descriptions as values. 
- q6_dic: dictionary with 'q6' and numerical codes (1 to 1684) as keys, and descriptions as values. 
- q7_dic: dictionary with 'q7' and numerical codes (1 to 5) as keys, and descriptions as values. 

Numerical codes 0 or 1 denote 'no' or 'yes' answers. Numerical codes starting with 1 denote descending categorical responses based on frequency (i.e., 1 meaning the most frequent answer). Question 'q6' is problematic because it has 1684 unique answers. Dictionaries 'q5_dic' and 'q6_dic' will not be made publicly available for confidentiality reason.

In [5]:
## Reading bookings_questions_data.csv as pandas dataframe questions_df.
questions_df = pd.read_csv('data/bookings_questions_data.csv', 
                           index_col='b_id', 
                           header=0, 
                           low_memory=False,
                           keep_default_na=False,
                           na_values='')

## Checking for completely empty rows.
if len(questions_df)!=len(questions_df.dropna(how='all')):
    print('Attention! There are completely empty rows!')
    
## Lower case for all.
for i in list(questions_df.columns):
    questions_df[i] = questions_df[i].str.lower()

## Fixing strings in rows that will become columns.
q_list = list(questions_df['b_extra_q'].value_counts(ascending=False).index)
questions_df['b_extra_q'].replace([q_list[2],q_list[4]],
                                  'q4 '+q_list[2], inplace=True)
questions_df['b_extra_q'].replace(q_list[0],
                                  'q5 '+q_list[0], inplace=True)
questions_df['b_extra_q'].replace(q_list[1],
                                  'q6 '+q_list[1], inplace=True)
questions_df['b_extra_q'].replace(q_list[3],
                                  'q7 '+q_list[3]+'?', inplace=True)

## Making each question a column with its answer.
questions_df = questions_df.pivot(columns='b_extra_q', values='b_extra_a')
questions_df.columns.name = None

## Coding questions q4-q7 and creating dictionary for them: q#_dic.
for i in list(questions_df.columns):
    if i.startswith('q4'):
        questions_df.loc[questions_df[i].str.startswith('yes', na=False),i]=1
        questions_df.loc[questions_df[i].str.startswith('no', na=False),i]=0
        questions_df[i] = questions_df[i].astype('Int64')
        q4_dic = {'question': i[3:], 0: 'no', 1: 'yes'}
        questions_df.rename(columns={i: i[0:2]}, inplace=True)
    elif i.startswith('q5'):
        j = list(questions_df[i].value_counts(ascending=False).index)
        jp = list(range(1, len(j)+1))
        zipped = list(zip(jp, j))
        zipped.insert(0, ('question', i[3:]))
        q5_dic = dict(zipped) 
        q5_indic = {value: key for key, value in q5_dic.items()}
        questions_df[i] = questions_df[i].map(q5_indic).astype('Int64')
        questions_df.rename(columns={i: i[0:2]}, inplace=True)
    elif i.startswith('q6'):
        j = list(questions_df[i].value_counts(ascending=False).index)
        jp = list(range(1, len(j)+1))
        zipped = list(zip(jp, j))
        zipped.insert(0, ('question', i[3:]))
        q6_dic = dict(zipped) 
        q6_indic = {value: key for key, value in q6_dic.items()}
        questions_df[i] = questions_df[i].map(q6_indic).astype('Int64')
        questions_df.rename(columns={i: i[0:2]}, inplace=True)
    elif i.startswith('q7'):
        j = list(questions_df[i].value_counts(ascending=False).index)
        jp = list(range(1, len(j)+1))
        zipped = list(zip(jp, j))
        zipped.insert(0, ('question', i[3:]))
        q7_dic = dict(zipped) 
        q7_indic = {value: key for key, value in q7_dic.items()}
        questions_df[i] = questions_df[i].map(q7_indic).astype('Int64')
        questions_df.rename(columns={i: i[0:2]}, inplace=True)
    else:
        print('Attention! Something is wrong!')

Next, we will work with 'bookings_addons_data.csv'. This file contains add-ons purchases associated with some bookings. It informs price, quantity and revenue (price x quantity) of 27 different add-ons. It also informs total quantity and revenue corresponding to all add-ons. The end product of the code cell will be:

- addons_df: dataframe with booking id and columns related to 'addon_01' through 'addon_27'.
- addons_dic: dictionary with 'addon_01' through 'addon_27' as keys, and their descriptions as values.

Add-ons can be, for example, souvenirs or special additions to the booking. The dictionary 'addons_dic' wil not be made publicly available for confidentiality reason.

In [6]:
## Reading bookings_addons_data.csv as pandas dataframe addons_df.
addons_df = pd.read_csv('data/bookings_addons_data.csv',
                        index_col='b_id', 
                        header=0,
                        low_memory=False,
                        keep_default_na=False,
                        na_values='')

## Dropping completely empty rows and fixing types.
addons_df.dropna(how='all', inplace=True)
addons_df.index = addons_df.index.map(int)
addons_df['b_addons_quant'] = addons_df['b_addons_quant'].astype('int64')

## Group by addons names.
agg_dic = {'b_addons_price':np.average,
           'b_addons_quant':np.sum,
           'b_addons_rev':np.sum}
addons_name_df = addons_df.groupby(['b_addons_name']).agg(agg_dic)

## Sort addons names first by quantity and then by revenue.
addons_name_df.sort_values(['b_addons_quant', 'b_addons_rev'],
                           ascending=[False, False],
                           inplace=True)

## Creating addons codes for addons names based on the previous sorting.
addons_name_df['b_addons_code'] = ['addon_'+f'{i:02d}' for i in 
                                   np.arange(1,len(addons_name_df)+1)]

## Creating dictionary to connect addons names to addons codes.
addons_dic = addons_name_df['b_addons_code'].to_dict()

## Adding a column of addons codes to the original dataframe addons_df.
addons_df['b_addons_code'] = addons_df['b_addons_name'].map(addons_dic)
addons_df = addons_df[['b_addons_code',
                       'b_addons_name',
                       'b_addons_price',
                       'b_addons_quant',
                       'b_addons_rev']]

## Remaking dictionary so addons codes give addons names if necessary.
addons_code_df = pd.DataFrame.from_dict(addons_dic.items())
addons_code_df.columns = ['b_addons_name','b_addons_code']
addons_code_df.set_index('b_addons_code', inplace=True)
addons_dic = addons_code_df['b_addons_name'].to_dict()

## Dropping column addons name for confidentiality reason.
addons_df.drop(columns=['b_addons_name'], inplace=True)

## Making each addon a column with its price, quantity and revenue.
addons_df = addons_df.pivot(columns='b_addons_code', 
                            values=['b_addons_price',
                                    'b_addons_quant',
                                    'b_addons_rev'])

## Flattening hierarchical columns.
addons_df.columns = [i[1]+'_'+i[0][9:]
                     for i in list(addons_df.columns)]

## Creating column with total quantity of addons.
col_list = list(filter(lambda x: x[7:8].isdigit() and x.endswith('_quant'),
                       addons_df.columns))
addons_df['tot_quant_addons_alt'] = addons_df[col_list].sum(axis=1, skipna=True)

## Creating column with total revenue from addons.
col_list = list(map(lambda x: x+'_rev', 
                    list(addons_dic.keys())))
addons_df['tot_rev_addons_alt'] = addons_df[col_list].sum(axis=1, skipna=True)

## Checking if price * quant = revenue for each addon.
for i in list(addons_dic.keys()):
    rev_a = addons_df[i+'_rev'].dropna().round(10)
    rev_b = addons_df[i+'_price']*addons_df[i+'_quant']
    rev_b = rev_b.dropna().round(10)
    if set(rev_a) != set(rev_b):
        print('Checking if price * quant = revenue for each addon:')
        print('Problem with', i)
        print('Is size the problem?', len(rev_a)!=len(rev_b))
        print('Where is the problem?')
        for j in zip(rev_a,rev_b):
            if j[0] != j[1]:
                print(f'{j[0]:0.20f}', f'{j[1]:0.20f}')

Finally, we will work with 'main_data.csv'. As the name suggests, this is the main file with several different information associate with all bookings between 2016-08-31 and 2017-12-31. There is information about the client booking a visit, the booking itself, and the firm to be visited. The end product of the code cell will be:

- main_df: dataframe with booking id and several columns of information. Some columns have self-explanatory titles. Other columns will have dictionaries with descriptions (see next items).
- firm_id_dic: dictionary with each firm's numerical code as key, and each firm's name as value.
- firm_type_dic: dictionary with numerical codes (1 to 3) as keys, and descriptions as values. 
- status_cancel_dic: dictionary with numerical codes (0 to 1) as keys, and descriptions as values.
- source_online_dic: dictionary with numerical codes (0 to 1) as keys, and descriptions as values.
- payment_cc_dic: dictionary with numerical codes (-1 to 1) as keys, and descriptions as values.
- discount_dic: dictionary with numerical codes (0 to 1) as keys, and descriptions as values.
- q1_bef_dic: dictionary with 'q1_bef' as key, and its description as value.
- q1_aft_dic: dictionary with 'q1_aft' as key, and its description as value.
- q2_dic: dictionary with 'q2' and numerical codes (0 to 3) as keys, and descriptions as values.
- q3_dic: dictionary with 'q3' and numerical codes (0 to 1) as keys, and descriptions as values.
    
Basically, for those columns with dictionaries, the reader can access '[insert column's title]_dic' to read descriptions. However, the dictionaries 'firm_id_dic' and 'firm_type_dic' will not be made publicly available for confidentiality reason.

In [7]:
## Reading main_data.csv as pandas dataframe main_df.
main_df = pd.read_csv('data/main_data.csv', 
                      index_col='b_id', 
                      header=0, 
                      low_memory=False,
                      keep_default_na=False,
                      na_values='')

## Checking for completely empty rows.
if len(main_df)!=len(main_df.dropna(how='all')):
    print('Attention! There are completely empty rows!')

## Fixing names of columns.
col_list = list(main_df.columns.values)
new_col_list = []
for i in col_list:
    if i[0:2] == 'b_':
        new_col_list.append(i[2:])
    elif i[0:2] == 'f_':
        new_col_list.append('firm_'+i[2:])
    elif i[0:2] == 'p_':
        new_col_list.append('visit_'+i[2:])
    else: print('Attention! Something is missing!')
main_df.columns = new_col_list
        
## Fixing types of some columns.
main_df['age'] = main_df['age'].astype('Int64')
main_df['guests_in'] = main_df['guests_in'].astype('Int64')
main_df['nps_bef'] = main_df['nps_bef'].astype('Int64')
main_df['nps_aft'] = main_df['nps_aft'].astype('Int64')
main_df['birthday'] = pd.to_datetime(main_df['birthday'])
main_df['date'] = pd.to_datetime(main_df['date'])
main_df.rename(columns={'date': 'contact_date'}, inplace=True)
main_df.insert(main_df.columns.get_loc('firm_loc')+1, 'contact_date',
               main_df.pop('contact_date'))

## Creating 'visit_datetime' from 'visit_date' and 'visit_time'.
main_df['visit_datetime'] = main_df['visit_date']+' '+main_df['visit_time']
main_df['visit_datetime'] = pd.to_datetime(main_df['visit_datetime'])
main_df.insert(main_df.columns.get_loc('visit_name')+1, 'visit_datetime',
               main_df.pop('visit_datetime'))
main_df.drop(columns=['visit_date', 'visit_time'], inplace=True) 

## Dropping some columns for confidentiality reasons.
main_df.drop(columns=['birthday', 'address', 'visit_name'], inplace=True)

## Creating dictionary with 'firm_id' keys and 'firm_name' values: firm_id_dic.
firm_name_df = main_df[['firm_id', 'firm_name']].groupby('firm_name').mean()
firm_name_dic = firm_name_df['firm_id'].to_dict()
firm_id_dic = {j: i for i, j in firm_name_dic.items()}
main_df.drop(columns='firm_name', inplace=True)

## Coding 'firm_type' and creating dictionary for it: firm_type_dic.
main_df['firm_type']=main_df['firm_type'].str.lower()
types_list = list(main_df['firm_type'].value_counts(ascending=False).index)
firm_type_dic = {1: types_list[0], 2: types_list[1], 3: types_list[2]}
firm_type_indic = {j: i for i, j in firm_type_dic.items()} 
main_df['firm_type'] = main_df['firm_type'].map(firm_type_indic)

## Coding 'status_cancel' and creating dictionary for it: status_cancel_dic.
main_df.rename(columns={'visit_status': 'status_cancel'}, inplace=True)
main_df['status_cancel'] = main_df['status_cancel'].str.lower()
status_cancel_dic = {0: 'confirmed', 1: 'cancelled'}
status_cancel_indic = {j: i for i, j in status_cancel_dic.items()} 
main_df['status_cancel'] = main_df['status_cancel'].map(status_cancel_indic)

## Coding 'source_online' and creating dictionary for it: source_online_dic.
main_df.rename(columns={'source': 'source_online'}, inplace=True)
main_df['source_online'] = main_df['source_online'].str.lower()
source_online_dic = {0: 'added', 1: 'online'}
source_online_indic = {j: i for i, j in source_online_dic.items()} 
main_df['source_online'] = main_df['source_online'].map(source_online_indic)

## Coding 'payment_cc' and creating dictionary for it: payment_cc_dic.
main_df.rename(columns={'payment': 'payment_cc'}, inplace=True)
main_df['payment_cc'] = main_df['payment_cc'].str.lower()
payment_cc_dic = {-1: 'none', 0: 'cash', 1: 'credit card'}
payment_cc_indic = {j: i for i, j in payment_cc_dic.items()} 
main_df['payment_cc'] = main_df['payment_cc'].map(payment_cc_indic)

## Coding 'discount' and creating dictionary for it: discount_dic.
main_df['discount'] = main_df['discount'].str.lower()
discount_dic = {0: 'no', 1: 'yes'}
main_df.loc[main_df['discount'].notna(), 'discount'] = 1
main_df.loc[main_df['discount'].isna(), 'discount'] = 0

## Organizing question 1 (q1_bef & q1_aft) and its dictionary (q1_bef_dic & q1_aft_dic).
q1_bef_dic = {'question': 'likelihood to recommend our product before visit on scale 0-10'} 
q1_aft_dic = {'question': 'likelihood to recommend our product after visit on scale 0-10'}
main_df.rename(columns = {'nps_bef': 'q1_bef', 'nps_aft': 'q1_aft'}, 
               inplace = True) 
main_df.insert(main_df.columns.get_loc('tot_rev')+1, 'q1_bef', main_df.pop('q1_bef'))
main_df.insert(main_df.columns.get_loc('q1_bef')+1, 'q1_aft', main_df.pop('q1_aft'))
main_df.drop(columns ='nps_q', inplace=True) 

## Organizing question 2 (q2) and its dictionary (q2_dic).
main_df['behavior_a'] = main_df['behavior_a'].str.lower()
main_df['behavior_a'].replace('at least once a year', 'occasionally', inplace=True)
q2_dic = {'question': 'how often did you use our product in the past year?', 
          0: 'never',
          1: 'occasionally',
          2: 'at least once a month', 
          3: 'at least once a week'}
q2_indic = {j: i for i, j in q2_dic.items()} 
main_df['q2'] = main_df['behavior_a'].map(q2_indic).astype('Int64')
main_df.insert(main_df.columns.get_loc('q1_aft')+1, 'q2', main_df.pop('q2'))
main_df.drop(columns =['behavior_q', 'behavior_a'], inplace=True) 

## Organizing question 3 (q3) and its dictionary (q3_dic).
q3_dic = {'question': 'would you join our email list?', 
          0: 'no', 
          1: 'yes'}
main_df.loc[main_df['email_a'].str.contains('I don', na=False), 'q3'] = 0
main_df.loc[main_df['email_a'].str.contains('events', na=False), 'q3'] = 1
main_df['q3'] = main_df['q3'].astype('Int64')
main_df.insert(main_df.columns.get_loc('q2')+1, 'q3', main_df.pop('q3'))
main_df.drop(columns =['email_q', 'email_a'], inplace=True)

Now, in the next code cell, we are going to:

1-) make a copy of 'main.df' called 'final_df'.  
2-) use the dictionary 'country_dic' on the dataframe 'final_df' to make the column 'country' show the full name of each country.  
3-) use the dictionaries 'us_region_dic', 'us_region_indic' and 'us_zipcode_dic' on the dataframe 'final_df' to create columns 'us_region' and 'us_zipcode'.  
4-) merge the dataframe 'questions_df' to the dataframe 'final_df', and organizing it.  
5-) merge the dataframe 'addons_df' to the dataframe 'final_df', and organizing it.  

The goal is to have all information included in a single dataframe: 'final_df'.  
All dictionaries will be preserved in 'final_dic' (a dictionary of dictionaries).  
All publicly available dictionaries will be preserved in 'final_dic_pub' (a dictionary of dictionaries).

In [8]:
## 1-) make a copy of 'main.df' called 'final_df'.
final_df = main_df.copy()

## 2-) use 'country_dic' on 'final_df'.
final_df['country'] = final_df['country'].map(country_dic)

## 3-) use 'us_region_dic', 'us_region_indic' and 'us_zipcode_dic' on 'final_df'.

## Creating column 'us_zipcode'.
f1 = final_df['country'] == 'united states'
f2 = final_df['country'].isna()
us_zip_list = []
for i in final_df.loc[f1 | f2, 'zipcode'].astype(str).values:
    i_c1 = i.strip().isdigit()
    i_c2 = len(i.strip()) <= 5
    i_c3 = '-' in i
    i_c4 = i.split('-')[0].strip().isdigit()
    i_c5 = len(i.split('-')[0].strip()) <= 5   
    if i_c1 and i_c2:
        j = f'{int(i.strip()):05d}'
        us_zip_list.append(j)
    elif i_c3 and i_c4 and i_c5:
        j = f'{int(i.split("-")[0].strip()):05d}'
        us_zip_list.append(j)
    else:
        j = np.nan
        us_zip_list.append(j)
final_df.loc[f1 | f2, 'us_zipcode'] = us_zip_list
final_df.insert(final_df.columns.get_loc('country')+1, 
                'us_zipcode', final_df.pop('us_zipcode'))

## Creating column 'us_region'.
f1 = final_df['country'] == 'united states'
f2 = final_df['country'].isna()
l1 = ['region', 'us_zipcode']
us_region_list = []
for i, j in final_df.loc[f1 | f2, l1].astype(str).values:
    i_c1 = i.strip().upper() in us_region_dic
    i_c2 = i.strip().lower() in us_region_indic
    j_c1 = str(j)[0:3] in us_zipcode_dic
    if i_c1 and j_c1:
        if i.strip().upper() == us_zipcode_dic[str(j)[0:3]]:
            k = i.strip().upper()
            us_region_list.append(k)
        else:
            k = 'unclear'
            us_region_list.append(k)
    elif i_c2 and j_c1:
        if us_region_indic[i.strip().lower()] == us_zipcode_dic[str(j)[0:3]]:
            k = us_region_indic[i.strip().lower()]
            us_region_list.append(k)
        else:
            k = 'unclear'
            us_region_list.append(k)
    elif i_c1:
        k = i.strip().upper()
        us_region_list.append(k)
    elif i_c2:
        k = us_region_indic[i.strip().lower()]
        us_region_list.append(k)
    elif j_c1:
        k = us_zipcode_dic[str(j)[0:3]]
        us_region_list.append(k)
    else:
        k = np.nan
        us_region_list.append(k)
final_df.loc[f1 | f2, 'us_region'] = us_region_list
final_df.insert(final_df.columns.get_loc('country')+1, 
                'us_region', final_df.pop('us_region'))

## Corrections and fine-tuning in columns 'country', 'us_region' and 'us_zipcode'.
f1 = final_df['country'].isna()
f2 = final_df['us_region'].notna()
f3 = final_df['us_region'] != 'International Mail'
f4 = final_df['us_region'] != 'NOT USED'
final_df.loc[f1 & f2 & f3 & f4, 'country'] = 'united states'

f1 = final_df['country'].isna()
f2 = final_df['us_region'] == 'International Mail'
f3 = final_df['us_region'] == 'NOT USED'
final_df.loc[f1 & (f2 | f3), 'country'] = 'japan'
final_df.loc[f1 & (f2 | f3), ['us_region', 'us_zipcode']] = [np.nan, np.nan] 

f1 = final_df['country'] == 'united states'
f2 = final_df['us_region'].isna()
f3 = final_df['us_region'] == 'International Mail'
f4 = final_df['us_region'] == 'NOT USED'
f5 = final_df['region'].notna()
f6 = final_df['zipcode'].notna()
final_df.loc[f1 & (f2 | f3 | f4) & (f5 | f6),
             ['country', 'us_region', 'us_zipcode']] = ['unclear', np.nan, np.nan]

f1 = final_df['country'].isna()
f2 = final_df['region'].notna()
f3 = final_df['zipcode'].notna()
final_df.loc[f1 & (f2 | f3), 'country'] = 'canada'

f1 = final_df['country'].notna()
f2 = final_df['country'] != 'united states'
f3 = final_df['country'] != 'unclear'
final_df.loc[f1 & f2 & f3, ['us_region', 'us_zipcode']] = ['foreigner','foreigner']

## Dropping now-unecessary columns for confidentiality reason.
final_df.drop(columns=['region', 'city', 'zipcode'], inplace=True)

## 4-) merge 'questions_df' to 'final_df'.
final_df = pd.merge(final_df, questions_df, on='b_id', how='outer', indicator=True)
final_df.rename(columns={'_merge': 'merge_questions'}, inplace=True)
final_df['merge_questions'] = final_df['merge_questions'].astype(str)

## 5-) merge 'addons_df' to 'final_df'.
final_df = pd.merge(final_df, addons_df, on='b_id', how='outer', indicator=True)
final_df.rename(columns={'_merge': 'merge_addons'}, inplace=True)
final_df['merge_addons'] = final_df['merge_addons'].astype(str)

## Filling up 'nan' in a column with available information in other column.
for i in ['tot_quant_addons','tot_rev_addons']:
    f1 = final_df[i].notna()
    f2 = final_df[i+'_alt'].isna()
    final_df.loc[f1 & f2, i+'_alt'] = final_df.loc[f1 & f2, i]
    f3 = final_df[i].isna()
    f4 = final_df[i+'_alt'].notna()
    final_df.loc[f3 & f4, i] = final_df.loc[f3 & f4, i+'_alt']
     
## Correcting 'tot_quant_addons'.
f1 = final_df['tot_quant_addons'] != final_df['tot_quant_addons_alt']
f2 = final_df['tot_rev_addons'] == final_df['tot_rev_addons_alt']
final_df.loc[f1 & f2, 'tot_quant_addons'] = final_df.loc[f1 & f2, 'tot_quant_addons_alt']

## Correcting 'tot_rev_addons'.
f1 = final_df['tot_quant_addons'] == final_df['tot_quant_addons_alt']
f2 = final_df['tot_rev_addons'] != final_df['tot_rev_addons_alt']
final_df.loc[f1 & f2, 'tot_rev_addons'] = final_df.loc[f1 & f2, 'tot_rev_addons_alt']

## Correcting 'tot_quant_addons_alt' and 'tot_rev_addons_alt'.
f1 = final_df['tot_quant_addons'] > final_df['tot_quant_addons_alt']
f2 = final_df['tot_rev_addons'] > final_df['tot_rev_addons_alt']
final_df.loc[f1 & f2, 'tot_quant_addons_alt'] = final_df.loc[f1 & f2, 'tot_quant_addons']
final_df.loc[f1 & f2, 'tot_rev_addons_alt'] = final_df.loc[f1 & f2, 'tot_rev_addons']

## Dropping now-unecessary columns and rounding 'tot_rev_addons'.
final_df.drop(columns=['tot_quant_addons_alt', 'tot_rev_addons_alt'], inplace=True)
final_df['tot_rev_addons'] = final_df['tot_rev_addons'].round(2)

## Correcting and rounding 'tot_rev'.
f1 = final_df['tot_rev'].isna()
f2 = final_df['tot_rev_addons'].notna()
f3 = final_df['tot_rev'] < final_df['tot_rev_addons']
final_df.loc[(f1 & f2) | f3, 'tot_rev'] = final_df.loc[(f1 & f2) | f3, 'tot_rev_addons']
final_df['tot_rev'] = final_df['tot_rev'].round(2)

## Recalculating and rounding 'fee'.
final_df['fee'] = (final_df['tot_rev'] - final_df['tot_rev_addons']) / final_df['guests']
final_df['fee'] = final_df['fee'].round(2)

## Creating 'final_dic' with all relevant dictionaries.
final_dic = {'country': country_dic,
             'us_region': us_region_dic, 
             'us_zipcode': us_zipcode_dic,
             'firm_id': firm_id_dic,
             'firm_type': firm_type_dic,
             'status_cancel': status_cancel_dic,
             'source_online': source_online_dic,
             'payment_cc': payment_cc_dic,
             'discount': discount_dic, 
             'q1_bef': q1_bef_dic, 
             'q1_aft': q1_aft_dic, 
             'q2': q2_dic, 
             'q3': q3_dic, 
             'q4': q4_dic, 
             'q5': q5_dic, 
             'q6': q6_dic, 
             'q7': q7_dic, 
             'addons': addons_dic}

## Creating 'final_dic_pub' with all public dictionaries.
final_dic_pub = {'country': country_dic,
                 'us_region': us_region_dic, 
                 'us_zipcode': us_zipcode_dic,
                 'firm_id': {'access denied': 'not publicly available'},
                 'firm_type': {'access denied': 'not publicly available'},
                 'status_cancel': status_cancel_dic,
                 'source_online': source_online_dic,
                 'payment_cc': payment_cc_dic,
                 'discount': discount_dic, 
                 'q1_bef': q1_bef_dic, 
                 'q1_aft': q1_aft_dic, 
                 'q2': q2_dic, 
                 'q3': q3_dic, 
                 'q4': q4_dic, 
                 'q5': {'access denied': 'not publicly available'}, 
                 'q6': {'access denied': 'not publicly available'}, 
                 'q7': q7_dic, 
                 'addons': {'access denied': 'not publicly available'}}

## Creating dataframe 'final_dic_df' from 'final_dic'.
final_dic_df = pd.DataFrame.from_dict(final_dic)

## Creating dataframe 'final_dic_pub_df' from 'final_dic_pub'
final_dic_pub_df = pd.DataFrame.from_dict(final_dic_pub)

print(final_df.info(verbose=True, null_counts=True))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56640 entries, 132755 to 295082
Data columns (total 112 columns):
age                 9051 non-null Int64
age_range           9051 non-null object
country             41135 non-null object
us_region           40599 non-null object
us_zipcode          40529 non-null object
firm_id             56425 non-null float64
firm_type           56425 non-null float64
firm_loc            56425 non-null object
contact_date        56425 non-null datetime64[ns]
visit_datetime      56425 non-null datetime64[ns]
status_cancel       56425 non-null float64
source_online       56425 non-null float64
payment_cc          56425 non-null float64
guests              56425 non-null float64
guests_in           34521 non-null Int64
fee                 56425 non-null float64
discount            56425 non-null float64
tot_quant_addons    56425 non-null float64
tot_rev_addons      56425 non-null float64
tot_rev             56425 non-null float64
q1_bef              3

In the next code cell, we are going to:

1-) Create directory 'data_new' where we are going to store csv files for analysis.  
2-) Save 'final_df.csv' and 'final_df_dtypelist.csv' in the new directory.  
3-) Save 'final_dic_df.csv' in the new directory.  
4-) Save 'final_dic_pub_df.csv' in the new directory.

'final_df.csv' and 'final_df_dtypelist.csv' will be used to recover 'final_df'.  
'final_dic_df.csv' will be used to recover 'final_dic'.  
'final_dic_pub_df.csv' will be used to recover 'final_dic_pub'.

Attention: once recovered, 'final_dic' and 'final_dic_pub' will have all their items as string.

In [9]:
## Saving csv files in 'data_new' for future analysis'.
if os.path.isdir('data_new') == False:
    os.mkdir('data_new')

files = [(final_df, 'data_new/final_df.csv'), 
         (final_dic_df, 'data_new/final_dic_df.csv'), 
         (final_dic_pub_df, 'data_new/final_dic_pub_df.csv')]

for file in files:
    file[0].to_csv(path_or_buf=file[1])
      
final_df_dtypedic = {}
for i in final_df.columns:
    i_dtype = str(final_df[i].dtype)
    final_df_dtypedic.update({i: i_dtype})
final_df_dtypelist = list(final_df_dtypedic.items())

with open('data_new/final_df_dtypelist.csv', 'w', newline='') as to_write:
    writer = csv.writer(to_write)
    writer.writerows(final_df_dtypelist)

In the next three code cells, we are going to demonstrate how to recover 'final_df', 'final_dic' and 'final_dic_pub' using the csv files in the 'data_new' directory . As said before, once recovered, 'final_dic' and 'final_dic_pub' will have all their items as string.

In [10]:
## Testing how to recover 'final_df'.
new_final_df = pd.read_csv('data_new/final_df.csv', 
                           index_col=0, 
                           header=0, 
                           low_memory=False,
                           keep_default_na=False,
                           na_values='')

with open('data_new/final_df_dtypelist.csv', newline='') as to_read:
    reader = csv.reader(to_read)
    data = list(reader)
    new_final_df_dtypedic = dict(data)

for i in new_final_df.columns:
    new_final_df[i] = new_final_df[i].astype(new_final_df_dtypedic[i])

print("Is 'new_final_df_dtypedic' equals to 'final_df_dtypedic'?")
print(new_final_df_dtypedic == final_df_dtypedic)
print("Is 'new_final_df' equals to 'final_df'?")
print(new_final_df.equals(final_df))

Is 'new_final_df_dtypedic' equals to 'final_df_dtypedic'?
True
Is 'new_final_df' equals to 'final_df'?
True


In [11]:
## Testing how to recover 'final_dic'.
new_final_dic_df = pd.read_csv('data_new/final_dic_df.csv',
                               index_col=0, 
                               header=0, 
                               low_memory=False,
                               keep_default_na=False,
                               na_values='')

new_final_dic = {}
for key in new_final_dic_df.columns:
    value = new_final_dic_df[key].dropna().to_dict()
    new_final_dic.update({key: value})

str_final_dic = {str(k): {str(i): str(j) for i, j in v.items()} 
                 for k, v in final_dic.items()}

print("Is 'new_final_dic' equals to 'final_dic' with all items as string?")
print(new_final_dic == str_final_dic)

Is 'new_final_dic' equals to 'final_dic' with all items as string?
True


In [12]:
## Testing how to recover 'final_dic_pub'.
new_final_dic_pub_df = pd.read_csv('data_new/final_dic_pub_df.csv',
                                   index_col=0, 
                                   header=0, 
                                   low_memory=False,
                                   keep_default_na=False,
                                   na_values='')

new_final_dic_pub = {}
for key in new_final_dic_pub_df.columns:
    value = new_final_dic_pub_df[key].dropna().to_dict()
    new_final_dic_pub.update({key: value})

str_final_dic_pub = {str(k): {str(i): str(j) for i, j in v.items()} 
                     for k, v in final_dic_pub.items()}

print("Is 'new_final_dic_pub' equals to 'final_dic_pub' with all items as string?")
print(new_final_dic_pub == str_final_dic_pub)

Is 'new_final_dic_pub' equals to 'final_dic_pub' with all items as string?
True
